<p style="align: center;"><img align=center src="https://mailfit.com/wp-content/uploads/2019/11/lego-5.png"  width=900></p>
<h1 style="text-align: center;"><b>«Домашняя работа» - Ускорение тестирования</b></h3>


## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd

import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

## Импорт данных

In [ ]:
data_path = "https://raw.githubusercontent.com/a-milenkin/Datasetes_for_Piplines/main/SkillBox/ab_speedup.csv"
data = pd.read_csv(data_path, index_col=0)
print("Размеры датасета", data.shape)
data.head(6)

Размеры датасета (50000, 2)


,group,metrica
strata,,
11,history,42.631346
6,history,14.844453
4,history,2.362768
37,history,79.494017
18,history,-22.627788
34,history,28.679378


<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой результаты некоторого А/B-Теста.

* `strata` - Некоторая информация по страту (группа), к которой относится пользователь.
* `group` - Группа эксперимента - тестовая, контрольная или прошлая история.
* `metrica` - Целивая метрика
    
</div>

In [ ]:
 data.reset_index()

,strata,group,metrica
0,11,history,42.631346
1,6,history,14.844453
2,4,history,2.362768
3,37,history,79.494017
4,18,history,-22.627788
...,...,...,...
49995,47,A,3.089373
49996,44,A,54.028849
49997,49,A,119.874609
49998,18,A,50.340589


<div class="alert alert-info">

<b>Задание:</b>    
    
Найти различие между группами, увеличив чувствительность тестов тремя способами поочередно:
* Бутстрап
* Стратификация
* CUPED
    
Вспомогательная статья Валерия Бабушкина про стратификацию и CUPED:

* habr.com/ru/company/yandex/blog/497804/

In [ ]:
group_a = data[data['group'] == 'A']
group_b = data[data['group'] == 'B']
boots_m = []
for _ in range(10000):
  boot_a = np.random.choice(group_a['metrica'], size = len(group_a), replace = True)
  boot_b = np.random.choice(group_b['metrica'], size = len(group_b), replace = True)
  boot_diff = np.mean(boot_a) - np.mean(boot_b)
  boots_m.append(boot_diff)
boots_m = np.array(boots_m)
p_value = np.sum(boots_m <= 0) / 10000
print(p_value)

0.006


разница найдена, среднее значение метрики по группе А отличается от среднего по группе Б

In [ ]:
data = data.reset_index()
def stratified_bootstrap(data, group_col, strat_col, value_col, n_iterations=1000):
    stratif = data[strat_col].unique()
    boot_diffs = []

    for _ in range(n_iterations):
        strat_samples = []

        for stratum in stratif:
            group_a = data[(data[group_col] == 'A') & (data[strat_col] == stratum)][value_col]
            group_b = data[(data[group_col] == 'B') & (data[strat_col] == stratum)][value_col]

            if len(group_a) > 0 and len(group_b) > 0:
                boot_a = np.random.choice(group_a, size=len(group_a), replace=True)
                boot_b = np.random.choice(group_b, size=len(group_b), replace=True)

                strat_samples.append(np.mean(boot_b) - np.mean(boot_a))

        boot_diffs.append(np.mean(strat_samples))

    return np.array(boot_diffs)
boot_diffs = stratified_bootstrap(data, group_col='group', strat_col='strata', value_col='metrica')


p_value = np.sum(boot_diffs >= 0) / len(boot_diffs)
print(f'p-value: {p_value}')


p-value: 0.028


Здесь также видно, что разница есть, так как p_value меньше 0.05

In [ ]:
from tqdm import tqdm


In [ ]:
data_history = data[data['group'] == 'history']
data_experiment = data[data['group'].isin(['A', 'B'])]

data_experiment = data_experiment.merge(data_history, on='strata', suffixes=('', '_history'))
data_experiment = data_experiment.rename(columns={'metrica_history': 'pre_metrica'})

theta = np.cov(data_experiment['metrica'], data_experiment['pre_metrica'])[0, 1] / np.var(data_experiment['pre_metrica'])
data_experiment['metrica_cuped'] = data_experiment['metrica'] - theta * (data_experiment['pre_metrica'] - data_experiment['pre_metrica'].mean())

def stratified_bootstrap_cuped(data, group_col, strat_col, value_col, n_iterations=20):
    strata = data[strat_col].unique()
    boot_diffs = []

    for _ in tqdm(range(n_iterations)):
        strat_samples = []

        for stratum in strata:
            group_a = data[(data[group_col] == 'A') & (data[strat_col] == stratum)][value_col]
            group_b = data[(data[group_col] == 'B') & (data[strat_col] == stratum)][value_col]

            if len(group_a) > 0 and len(group_b) > 0:
                boot_a = np.random.choice(group_a, size=1000, replace=True)
                boot_b = np.random.choice(group_b, size=1000, replace=True)

                strat_samples.append(np.mean(boot_b) - np.mean(boot_a))

        boot_diffs.append(np.mean(strat_samples))

    return np.array(boot_diffs)

boot_diffs_cuped = stratified_bootstrap_cuped(data_experiment, group_col='group', strat_col='strata', value_col='metrica_cuped')

p_value_cuped = np.sum(boot_diffs_cuped >= 0) / len(boot_diffs_cuped)
print(f'p-value: {p_value_cuped}')

  0%|          | 0/20 [00:00<?, ?it/s]